In [ ]:
!pip install AutoGluon
!pip install filterpy

In [3]:
import random
import pandas as pd
import numpy as np
import os
import glob
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise
from tqdm import tqdm
import joblib
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error
from scipy.signal import butter, lfilter

# F.E

In [ ]:
def preprocessing(input_path):
  all_input_list = sorted(glob.glob(input_path))
  train = pd.DataFrame()
  for datapath in all_input_list:
    data = pd.read_csv(datapath) # 이게 지금 28일치 * 24시
    data['obs_time'] = data.index % 24 # 시간통일
    data = abs(data)
    col_list = data.columns
    for i in range(0,len(col_list)):
      col = col_list[i]    
      if '누적' in col :
        data[col] = data.groupby((data.obs_time == 0).cumsum()).agg(col_list[i-1]).cumsum()
    data.to_csv(datapath,index=False)
    train = pd.concat([train,data])
  return train
input_path_train = '/content/drive/MyDrive/상추/original_data/train_input/*.csv'
input_path_test = '/content/drive/MyDrive/상추/original_data/test_input/*.csv'

preprocessing(input_path_train)
preprocessing(input_path_test)

train_input_list = sorted(glob.glob('/content/drive/MyDrive/상추/original_data/train_input/*.csv'))
train_target_list = sorted(glob.glob('/content/drive/MyDrive/상추/original_data/train_target/*.csv'))

test_input_list = sorted(glob.glob('/content/drive/MyDrive/상추/original_data/test_input/*.csv'))
test_target_list = sorted(glob.glob('/content/drive/MyDrive/상추/original_data/test_target/*.csv'))
def make_dataset(all_input_list, all_target_list):
    df_all = pd.DataFrame()
    length = len(all_input_list)
    for idx in range(length):
        X = pd.read_csv(all_input_list[idx])
        y = pd.read_csv(all_target_list[idx])
        y['DAT'] = y['DAT']-1
        df_concat = pd.merge(X, y, on='DAT', how='left')
        df_concat['Case'] = idx+1
        df_all = pd.concat([df_all, df_concat])
    return df_all

train = make_dataset(train_input_list, train_target_list)
test = make_dataset(test_input_list, test_target_list)

train = train.sort_values(by=['Case','DAT'], axis=0).reset_index(drop=True)
test = test.sort_values(by=['Case','DAT'], axis=0).reset_index(drop=True)

def filtering(df):

    f1 = df[df['co2관측치']>578]['Case'].unique().tolist()
    f2 = df[((df['내부습도관측치']<74.4)&(df['내부습도관측치']>73.6))|((df['내부습도관측치']<30)&(df['내부습도관측치']>27))|((df['내부습도관측치']<54.5)&(df['내부습도관측치']>54.4)) ]['Case'].unique().tolist()
    f3 = df[(df['일간누적분무량']<150)]['Case'].unique().tolist()
    
    d1 = df[(df['시간당분무량']>2100)]['Case'].unique().tolist()
    d2 = df[(df['내부온도관측치']>31.124)&(df['내부온도관측치']<31.126)]['Case'].unique().tolist()
    d3 = df[(df['co2관측치']<510.1)&(df['co2관측치']>=510)]['Case'].unique().tolist()
    d4 = df[(df['시간당백색광량']>3094.0)&(df['시간당백색광량']<3094.2)]['Case'].unique().tolist()
    d5 = df[(df['시간당백색광량']>13400)&(df['시간당백색광량']<13400.5472)]['Case'].unique().tolist()

    return_arr = list(set(f1)&set(f2)&set(f3))
    for i in set(d1+d2+d3+d4+d5):
        if i in return_arr :
            return_arr.remove(i)

    return return_arr

trainfilter = filtering(train)
testfilter = filtering(test)

train['내부온도관측치'][train['내부온도관측치'] == 0] = train['내부온도관측치'].mean()
train['내부습도관측치'][train['내부습도관측치'] == 0] = train['내부습도관측치'].mean()
train['05시내부온도관측치누적'] = 0
train['19시내부온도관측치누적'] = 0
train['23시내부온도관측치누적'] = 0
train['05시내부습도관측치누적'] = 0
train['19시내부습도관측치누적'] = 0
train['23시내부습도관측치누적'] = 0
train['05시co2관측치누적'] = 0
train['19시co2관측치누적'] = 0
train['23시co2관측치누적'] = 0
train['05시ec관측치누적'] = 0
train['19시ec관측치누적'] = 0
train['23시ec관측치누적'] = 0
train['05시분무량누적'] = 0
train['19시분무량누적'] = 0
train['23시분무량누적'] = 0
train['05시백색광누적'] = 0
train['19시백색광누적'] = 0
train['23시백색광누적'] = 0
train['05시적색광누적'] = 0
train['19시적색광누적'] = 0
train['23시적색광누적'] = 0
train['05시청색광누적'] = 0
train['19시청색광누적'] = 0
train['23시청색광누적'] = 0
train['05시총광량누적'] = 0
train['19시총광량누적'] = 0
train['23시총광량누적'] = 0
test['05시내부온도관측치누적'] = 0
test['19시내부온도관측치누적'] = 0
test['23시내부온도관측치누적'] = 0
test['05시내부습도관측치누적'] = 0
test['19시내부습도관측치누적'] = 0
test['23시내부습도관측치누적'] = 0
test['05시co2관측치누적'] = 0
test['19시co2관측치누적'] = 0
test['23시co2관측치누적'] = 0
test['05시ec관측치누적'] = 0
test['19시ec관측치누적'] = 0
test['23시ec관측치누적'] = 0
test['05시분무량누적'] = 0
test['19시분무량누적'] = 0
test['23시분무량누적'] = 0
test['05시백색광누적'] = 0
test['19시백색광누적'] = 0
test['23시백색광누적'] = 0
test['05시적색광누적'] = 0
test['19시적색광누적'] = 0
test['23시적색광누적'] = 0
test['05시청색광누적'] = 0
test['19시청색광누적'] = 0
test['23시청색광누적'] = 0
test['05시총광량누적'] = 0
test['19시총광량누적'] = 0
test['23시총광량누적'] = 0

train = train.drop(train.filter(regex='일간누적').columns, axis=1)
test = test.drop(test.filter(regex='일간누적').columns, axis=1)

train_new = train
test_new = test

train['시간대'] = 0
train['시간대'][(train['obs_time'] >= 0) & (train['obs_time'] <= 5)] = 1
train['시간대'][(train['obs_time'] > 5) & (train['obs_time'] < 20)] = 2
train['시간대'][(train['obs_time'] >= 20) & (train['obs_time'] <= 23)] = 3
test['시간대'] = 0
test['시간대'][(test['obs_time'] >= 0) & (test['obs_time'] <= 5)] = 1
test['시간대'][(test['obs_time'] > 5) & (test['obs_time'] < 20)] = 2
test['시간대'][(test['obs_time'] >= 20) & (test['obs_time'] <= 23)] = 3

train['hypothesis1'] = 0
for i in range(22,len(train),24):
  s = train.iloc[i,6] + train.iloc[i+1,6] * ((train.iloc[i, 3] + train.iloc[i+1,3]) / 2)
  train.iloc[i, 41] = s
test['hypothesis1'] = 0
for i in range(22,len(test),24):
  s = test.iloc[i,6] + test.iloc[i+1,6] * ((test.iloc[i, 3] + test.iloc[i+1,3]) / 2)
  test.iloc[i, 41] = s
train['hypothesis2'] = 0
for i in range(22,len(train),24):
  s = train.iloc[i+1,6] * (train.iloc[i+1,3])
  train.iloc[i, 42] = s
test['hypothesis2'] = 0
for i in range(22,len(test),24):
  s = test.iloc[i+1,6] * (test.iloc[i+1,3])
  test.iloc[i, 42] = s
train['hypothesis3'] = 0
for i in range(22,len(train),24):
  s = (train.iloc[i,6] + train.iloc[i+1,6]) * ((train.iloc[i, 3] + train.iloc[i+1,3]) / 2)
  train.iloc[i, 43] = s
test['hypothesis3'] = 0
for i in range(22,len(test),24):
  s = (test.iloc[i,6] + test.iloc[i+1,6]) * ((test.iloc[i, 3] + test.iloc[i+1,3]) / 2)
  test.iloc[i, 43] = s

case_list = train['Case'].unique()
for c in case_list : 
    if c in trainfilter :
        continue
    train.loc[(train['Case'] == c ), 'hypothesis1'] = 0
    train.loc[(train['Case'] == c ), 'hypothesis2'] = 0
    train.loc[(train['Case'] == c ), 'hypothesis3'] = 0
case_list = test['Case'].unique()
for c in case_list : 
    if c in testfilter :
        continue
    test.loc[(test['Case'] == c ), 'hypothesis1'] = 0
    test.loc[(test['Case'] == c ), 'hypothesis2'] = 0
    test.loc[(test['Case'] == c ), 'hypothesis3'] = 0

train_x = train.drop(['predicted_weight_g'], axis = 1)
train_y = train['predicted_weight_g']
test_x = test.drop(['predicted_weight_g'], axis=1)
train_x = train_x.groupby(['DAT','Case','시간대']).sum().reset_index()
train_y_ready = train.groupby(['DAT','Case','시간대']).mean().reset_index()
test_x = test_x.groupby(['DAT','Case','시간대']).sum().reset_index()
train_x = train_x.sort_values(by=['Case','DAT','시간대'], axis=0).reset_index()
train_y_ready = train_y_ready.sort_values(by=['Case','DAT','시간대'], axis=0).reset_index()
test_x = test_x.sort_values(by=['Case','DAT','시간대'], axis=0).reset_index()
train_x.drop(['index'], axis = 1, inplace=True)
test_x.drop(['index'], axis = 1, inplace=True)
x = train_x.groupby(['Case'])['내부온도관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
x.drop(['level_1'], axis = 1, inplace = True)
train_x['내부온도관측치'] = x
x2 = train_x.groupby(['Case'])['내부습도관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
x2.drop(['level_1'], axis = 1, inplace = True)
train_x['내부습도관측치'] = x2
x3 = train_x.groupby(['Case'])['co2관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
x3.drop(['level_1'], axis = 1, inplace = True)
train_x['co2관측치'] = x3
x4 = train_x.groupby(['Case'])['ec관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
x4.drop(['level_1'], axis = 1, inplace = True)
train_x['ec관측치'] = x4
x5 = train_x.groupby(['Case'])['시간당분무량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
x5.drop(['level_1'], axis = 1, inplace = True)
train_x['시간당분무량'] = x5
x6 = train_x.groupby(['Case'])['시간당백색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
x6.drop(['level_1'], axis = 1, inplace = True)
train_x['시간당백색광량'] = x6
x7 = train_x.groupby(['Case'])['시간당적색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
x7.drop(['level_1'], axis = 1, inplace = True)
train_x['시간당적색광량'] = x7
x8 = train_x.groupby(['Case'])['시간당청색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
x8.drop(['level_1'], axis = 1, inplace = True)
train_x['시간당청색광량'] = x8
x9 = train_x.groupby(['Case'])['시간당총광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
x9.drop(['level_1'], axis = 1, inplace = True)
train_x['시간당총광량'] = x9
t = test_x.groupby(['Case'])['내부온도관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
t.drop(['level_1'], axis = 1, inplace = True)
test_x['내부온도관측치'] = t
t2 = test_x.groupby(['Case'])['내부습도관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
t2.drop(['level_1'], axis = 1, inplace = True)
test_x['내부습도관측치'] = t2
t3 = test_x.groupby(['Case'])['co2관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
t3.drop(['level_1'], axis = 1, inplace = True)
test_x['co2관측치'] = t3
t4 = test_x.groupby(['Case'])['ec관측치'].expanding().sum().reset_index().drop(['Case'], axis = 1)
t4.drop(['level_1'], axis = 1, inplace = True)
test_x['ec관측치'] = t4
t5 = test_x.groupby(['Case'])['시간당분무량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
t5.drop(['level_1'], axis = 1, inplace = True)
test_x['시간당분무량'] = t5
t6 = test_x.groupby(['Case'])['시간당백색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
t6.drop(['level_1'], axis = 1, inplace = True)
test_x['시간당백색광량'] = t6
t7 = test_x.groupby(['Case'])['시간당적색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
t7.drop(['level_1'], axis = 1, inplace = True)
test_x['시간당적색광량'] = t7
t8 = test_x.groupby(['Case'])['시간당청색광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
t8.drop(['level_1'], axis = 1, inplace = True)
test_x['시간당청색광량'] = t8
t9 = test_x.groupby(['Case'])['시간당총광량'].expanding().sum().reset_index().drop(['Case'], axis = 1)
t9.drop(['level_1'], axis = 1, inplace = True)
test_x['시간당총광량'] = t9
train_new_x = train_new.drop(['predicted_weight_g'], axis = 1)
train_new_y = train_new['predicted_weight_g']
test_new_x = test_new.drop(['predicted_weight_g'], axis=1)
train_new_x = train_new_x.groupby(['DAT','Case']).sum().reset_index()
train_new_y_ready = train_new.groupby(['DAT','Case']).mean().reset_index()
test_new_x = test_new_x.groupby(['DAT','Case']).sum().reset_index()
train_new_x = train_new_x.sort_values(by=['Case','DAT'], axis=0).reset_index()
train_new_y_ready = train_new_y_ready.sort_values(by=['Case','DAT',], axis=0).reset_index()
test_new_x = test_new_x.sort_values(by=['Case','DAT'], axis=0).reset_index()
train_new_x.drop(['index'], axis = 1, inplace=True)
test_new_x.drop(['index'], axis = 1, inplace=True)
for i in range(len(train_new_x)):
  train_new_x.iloc[i,12] = train_x.iloc[0 + (3*i),4]
  train_new_x.iloc[i,13] = train_x.iloc[1 + (3*i),4]
  train_new_x.iloc[i,14] = train_x.iloc[2 + (3*i),4]
  train_new_x.iloc[i,15] = train_x.iloc[0 + (3*i),5]
  train_new_x.iloc[i,16] = train_x.iloc[1 + (3*i),5]
  train_new_x.iloc[i,17] = train_x.iloc[2 + (3*i),5]
  train_new_x.iloc[i,18] = train_x.iloc[0 + (3*i),6]
  train_new_x.iloc[i,19] = train_x.iloc[1 + (3*i),6]
  train_new_x.iloc[i,20] = train_x.iloc[2 + (3*i),6]
  train_new_x.iloc[i,21] = train_x.iloc[0 + (3*i),7]
  train_new_x.iloc[i,22] = train_x.iloc[1 + (3*i),7]
  train_new_x.iloc[i,23] = train_x.iloc[2 + (3*i),7]
  train_new_x.iloc[i,24] = train_x.iloc[0 + (3*i),8]
  train_new_x.iloc[i,25] = train_x.iloc[1 + (3*i),8]
  train_new_x.iloc[i,26] = train_x.iloc[2 + (3*i),8]
  train_new_x.iloc[i,27] = train_x.iloc[0 + (3*i),9]
  train_new_x.iloc[i,28] = train_x.iloc[1 + (3*i),9]
  train_new_x.iloc[i,29] = train_x.iloc[2 + (3*i),9]
  train_new_x.iloc[i,30] = train_x.iloc[0 + (3*i),10]
  train_new_x.iloc[i,31] = train_x.iloc[1 + (3*i),10]
  train_new_x.iloc[i,32] = train_x.iloc[2 + (3*i),10]
  train_new_x.iloc[i,33] = train_x.iloc[0 + (3*i),11]
  train_new_x.iloc[i,34] = train_x.iloc[1 + (3*i),11]
  train_new_x.iloc[i,35] = train_x.iloc[2 + (3*i),11]
  train_new_x.iloc[i,36] = train_x.iloc[0 + (3*i),12]
  train_new_x.iloc[i,37] = train_x.iloc[1 + (3*i),12]
  train_new_x.iloc[i,38] = train_x.iloc[2 + (3*i),12]
for i in range(len(test_new_x)):
  test_new_x.iloc[i,12] = test_x.iloc[0 + (3*i),4]
  test_new_x.iloc[i,13] = test_x.iloc[1 + (3*i),4]
  test_new_x.iloc[i,14] = test_x.iloc[2 + (3*i),4]
  test_new_x.iloc[i,15] = test_x.iloc[0 + (3*i),5]
  test_new_x.iloc[i,16] = test_x.iloc[1 + (3*i),5]
  test_new_x.iloc[i,17] = test_x.iloc[2 + (3*i),5]
  test_new_x.iloc[i,18] = test_x.iloc[0 + (3*i),6]
  test_new_x.iloc[i,19] = test_x.iloc[1 + (3*i),6]
  test_new_x.iloc[i,20] = test_x.iloc[2 + (3*i),6]
  test_new_x.iloc[i,21] = test_x.iloc[0 + (3*i),7]
  test_new_x.iloc[i,22] = test_x.iloc[1 + (3*i),7]
  test_new_x.iloc[i,23] = test_x.iloc[2 + (3*i),7]
  test_new_x.iloc[i,24] = test_x.iloc[0 + (3*i),8]
  test_new_x.iloc[i,25] = test_x.iloc[1 + (3*i),8]
  test_new_x.iloc[i,26] = test_x.iloc[2 + (3*i),8]
  test_new_x.iloc[i,27] = test_x.iloc[0 + (3*i),9]
  test_new_x.iloc[i,28] = test_x.iloc[1 + (3*i),9]
  test_new_x.iloc[i,29] = test_x.iloc[2 + (3*i),9]
  test_new_x.iloc[i,30] = test_x.iloc[0 + (3*i),10]
  test_new_x.iloc[i,31] = test_x.iloc[1 + (3*i),10]
  test_new_x.iloc[i,32] = test_x.iloc[2 + (3*i),10]
  test_new_x.iloc[i,33] = test_x.iloc[0 + (3*i),11]
  test_new_x.iloc[i,34] = test_x.iloc[1 + (3*i),11]
  test_new_x.iloc[i,35] = test_x.iloc[2 + (3*i),11]
  test_new_x.iloc[i,36] = test_x.iloc[0 + (3*i),12]
  test_new_x.iloc[i,37] = test_x.iloc[1 + (3*i),12]
  test_new_x.iloc[i,38] = test_x.iloc[2 + (3*i),12]
train_new_y = train_new_y_ready['predicted_weight_g']
train_new_x.drop(['obs_time','시간대','내부온도관측치','내부습도관측치','co2관측치','ec관측치'], axis = 1, inplace = True)
test_new_x.drop(['obs_time','시간대','내부온도관측치','내부습도관측치','co2관측치','ec관측치'], axis = 1, inplace = True)
train_x = train_new_x
train_y = train_new_y
test_x = test_new_x
train_x = train_x.drop(train_x.filter(regex='시간당').columns, axis=1)
test_x = test_x.drop(test_x.filter(regex='시간당').columns, axis=1)

train_x['하루평균온도'] = (train_x['05시내부온도관측치누적'] + train_x['19시내부온도관측치누적'] + train_x['23시내부온도관측치누적']) / 3
train_x['하루평균습도'] = (train_x['05시내부습도관측치누적'] + train_x['19시내부습도관측치누적'] + train_x['23시내부습도관측치누적']) / 3
train_x['하루평균co2'] = (train_x['05시co2관측치누적'] + train_x['19시co2관측치누적'] + train_x['23시co2관측치누적']) / 3
train_x['하루평균ec'] = (train_x['05시ec관측치누적'] + train_x['19시ec관측치누적'] + train_x['23시ec관측치누적']) / 3
train_x['하루평균분무량'] = (train_x['05시분무량누적'] + train_x['19시분무량누적'] + train_x['23시분무량누적']) / 3
train_x['하루평균백색광'] = (train_x['05시백색광누적'] + train_x['19시백색광누적'] + train_x['23시백색광누적']) / 3
train_x['하루평균적색광'] = (train_x['05시적색광누적'] + train_x['19시적색광누적'] + train_x['23시적색광누적']) / 3
train_x['하루평균청색광'] = (train_x['05시청색광누적'] + train_x['19시청색광누적'] + train_x['23시청색광누적']) / 3
train_x['하루평균총광량'] = (train_x['05시총광량누적'] + train_x['19시총광량누적'] + train_x['23시총광량누적']) / 3
test_x['하루평균온도'] = (test_x['05시내부온도관측치누적'] + test_x['19시내부온도관측치누적'] + test_x['23시내부온도관측치누적']) / 3
test_x['하루평균습도'] = (test_x['05시내부습도관측치누적'] + test_x['19시내부습도관측치누적'] + test_x['23시내부습도관측치누적']) / 3
test_x['하루평균co2'] = (test_x['05시co2관측치누적'] + test_x['19시co2관측치누적'] + test_x['23시co2관측치누적']) / 3
test_x['하루평균ec'] = (test_x['05시ec관측치누적'] + test_x['19시ec관측치누적'] + test_x['23시ec관측치누적']) / 3
test_x['하루평균분무량'] = (test_x['05시분무량누적'] + test_x['19시분무량누적'] + test_x['23시분무량누적']) / 3
test_x['하루평균백색광'] = (test_x['05시백색광누적'] + test_x['19시백색광누적'] + test_x['23시백색광누적']) / 3
test_x['하루평균적색광'] = (test_x['05시적색광누적'] + test_x['19시적색광누적'] + test_x['23시적색광누적']) / 3
test_x['하루평균청색광'] = (test_x['05시청색광누적'] + test_x['19시청색광누적'] + test_x['23시청색광누적']) / 3
test_x['하루평균총광량'] = (test_x['05시총광량누적'] + test_x['19시총광량누적'] + test_x['23시총광량누적']) / 3
train_x = train_x.drop(train_x.filter(regex = '총광').columns, axis =1)
test_x = test_x.drop(test_x.filter(regex = '총광').columns, axis =1)
train_x['hypothesis4'] = (train_x['05시ec관측치누적']+1) * (train_x['05시분무량누적']+1)
train_x['hypothesis5'] = (train_x['19시ec관측치누적']+1) * (train_x['19시분무량누적']+1)
train_x['hypothesis6'] = (train_x['23시ec관측치누적']+1) * (train_x['23시분무량누적']+1)
train_x['hypothesis7'] = (train_x['하루평균ec']+1) * (train_x['하루평균분무량']+1)
test_x['hypothesis4'] = (test_x['05시ec관측치누적']+1) * (test_x['05시분무량누적']+1)
test_x['hypothesis5'] = (test_x['19시ec관측치누적']+1) * (test_x['19시분무량누적']+1)
test_x['hypothesis6'] = (test_x['23시ec관측치누적']+1) * (test_x['23시분무량누적']+1)
test_x['hypothesis7'] = (test_x['하루평균ec']+1) * (test_x['하루평균분무량']+1)
train_x['적색_+_청색05'] = (train_x['05시적색광누적']) + (train_x['05시청색광누적'])
train_x['적색_+_청색19'] = (train_x['19시적색광누적']) + (train_x['19시청색광누적'])
train_x['적색_+_청색23'] = (train_x['23시적색광누적']) + (train_x['23시청색광누적'])
train_x['적색_+_청색평균'] = (train_x['하루평균적색광']) + (train_x['하루평균청색광'])
test_x['적색_+_청색05'] = (test_x['05시적색광누적']) + (test_x['05시청색광누적'])
test_x['적색_+_청색19'] = (test_x['19시적색광누적']) + (test_x['19시청색광누적'])
test_x['적색_+_청색23'] = (test_x['23시적색광누적']) + (test_x['23시청색광누적'])
test_x['적색_+_청색평균'] = (test_x['하루평균적색광']) + (test_x['하루평균청색광'])
train_x['hypothesis123'] =  train_x['hypothesis1'] + train_x['hypothesis2'] + train_x['hypothesis3']
test_x['hypothesis123'] =  test_x['hypothesis1'] + test_x['hypothesis2'] + test_x['hypothesis3']
train_x['hypothesis12'] =  train_x['hypothesis2'] + train_x['hypothesis1']
test_x['hypothesis12'] =  test_x['hypothesis2'] + test_x['hypothesis1']
train_x['hypothesis23'] =  train_x['hypothesis2'] + train_x['hypothesis3']
test_x['hypothesis23'] =  test_x['hypothesis2'] + test_x['hypothesis3']
train_x['hypothesis13'] = train_x['hypothesis1'] + train_x['hypothesis3']
test_x['hypothesis13'] =  test_x['hypothesis1'] + test_x['hypothesis3']
train_x.drop(train_x.filter(regex = '백색'), axis = 1, inplace=True)
test_x.drop(test_x.filter(regex = '백색'), axis = 1, inplace=True)
train_x.drop(train_x.filter(regex = '청색'), axis = 1, inplace=True)
test_x.drop(test_x.filter(regex = '청색'), axis = 1, inplace=True)
train_x.drop(['hypothesis1','hypothesis2','hypothesis3'],axis=1,inplace=True)
test_x.drop(['hypothesis1','hypothesis2','hypothesis3'],axis=1,inplace=True)
basic_col = train_x.columns
kal_train = train_x
for i in range(2,34):
  train_x['kf_X_'+str(i)]=0
case_list = train_x['Case'].unique()
for j in case_list:
  kal = train_x[train_x['Case'] == j]
  for i in (range(len(basic_col))):
      if((i == 0) | (i == 1)):
        continue
      current=0
      sum_c=[]
      z = kal.loc[:, kal.columns[i]]
      a = []        
      b = []     
      my_filter = KalmanFilter(dim_x=2,dim_z=1) 
      my_filter.x = np.array([[2.],[0.]])       
      my_filter.F = np.array([[1.,1.], [0.,1.]])   
      my_filter.H = np.array([[1.,0.]])
      my_filter.P *= 1000.                
      my_filter.R = 5                     
      my_filter.Q = Q_discrete_white_noise(dim = 2,dt=.1,var=.5) 
      for k in z.values:
          my_filter.predict()
          my_filter.update(k)
          x = my_filter.x
          a.extend(x[0])
          b.append(k)
      sum_c=sum_c+a
      train_x['kf_X_'+str(i)][train_x['Case'] == j] = sum_c

basic_col = test_x.columns
kal_test = test_x
for i in range(2,34):
  test_x['kf_X_'+str(i)]=0
case_list = test_x['Case'].unique()
for j in case_list:
  kal = test_x[test_x['Case'] == j]
  for i in (range(len(basic_col))):
      if((i == 0) | (i == 1)):
        continue
      current=0
      sum_c=[]
      z = kal.loc[:, kal.columns[i]]
      a = []           
      b = []           
      my_filter = KalmanFilter(dim_x=2,dim_z=1) 
      my_filter.x = np.array([[2.],[0.]])      
      my_filter.F = np.array([[1.,1.], [0.,1.]])   
      my_filter.H = np.array([[1.,0.]])  
      my_filter.P *= 1000.              
      my_filter.R = 5                   
      my_filter.Q = Q_discrete_white_noise(dim = 2,dt=.1,var=.5)
      for k in z.values:
          my_filter.predict()
          my_filter.update(k)
          x = my_filter.x
          a.extend(x[0])
          b.append(k)
      sum_c=sum_c+a
      test_x['kf_X_'+str(i)][test_x['Case'] == j] = sum_c

copy_train_x = train_x.copy()
copy_test_x = test_x.copy()
copy_train_x.drop(copy_train_x.filter(regex = 'kf').columns, axis = 1, inplace = True)
copy_test_x.drop(copy_test_x.filter(regex = 'kf').columns, axis = 1, inplace = True)
raw_cols_tr = copy_train_x.columns
raw_cols_te = copy_test_x.columns
new_train_x = pd.DataFrame()

case_list = copy_train_x['Case'].unique()
for i in case_list:
  target = copy_train_x[copy_train_x['Case'] == i]
  mean_arr = []
  median_arr = []
  pbar = (target.columns[:])
  for column in pbar:
      column_list = target[column].to_list()
      for i in range(7):
        if((i) == (len(column_list))):
            break
        mean_arr.append(column_list[i])
        median_arr.append(column_list[i])
      for i in range(7, len(column_list)):
          mean_arr.append(float(np.mean(column_list[i-7:i])))
          median_arr.append(float(np.median(column_list[i-7:i])))
      target[f'{column}_mean_7'] = mean_arr
      target[f'{column}_median_7'] = median_arr
      mean_arr = []
      median_arr = []
  new_train_x = pd.concat([new_train_x, target], axis=0)
train_x_moving_7 = new_train_x.drop(raw_cols_tr, axis=1)

new_test_x = pd.DataFrame()
case_list = copy_test_x['Case'].unique()
for i in case_list:
  target = copy_test_x[copy_test_x['Case'] == i]
  mean_arr = []
  median_arr = []
  pbar = (target.columns[:])
  for column in pbar:
      column_list = target[column].to_list()
      for i in range(7):
        if((i) == (len(column_list))):
            break
        mean_arr.append(column_list[i])
        median_arr.append(column_list[i])
      for i in range(7, len(column_list)):
          mean_arr.append(float(np.mean(column_list[i-7:i])))
          median_arr.append(float(np.median(column_list[i-7:i])))
      target[f'{column}_mean_7'] = mean_arr
      target[f'{column}_median_7'] = median_arr
      mean_arr = []
      median_arr = []
  new_test_x = pd.concat([new_test_x, target], axis=0)
test_x_moving_7 = new_test_x.drop(raw_cols_te, axis=1)
train_x = pd.concat([train_x, train_x_moving_7], axis=1)
test_x = pd.concat([test_x, test_x_moving_7], axis=1)

raw_cols_tr = copy_train_x.columns
raw_cols_te = copy_test_x.columns
new_train_x = pd.DataFrame()
case_list = copy_train_x['Case'].unique()
for i in case_list:
  target = copy_train_x[copy_train_x['Case'] == i]
  mean_arr = []
  median_arr = []
  pbar = (target.columns[:])
  for column in pbar:
      column_list = target[column].to_list()
      for i in range(14):
        if((i) == (len(column_list))):
            break
        mean_arr.append(column_list[i])
        median_arr.append(column_list[i])
      for i in range(14, len(column_list)):
          mean_arr.append(float(np.mean(column_list[i-14:i])))
          median_arr.append(float(np.median(column_list[i-14:i])))
      target[f'{column}_mean_14'] = mean_arr
      target[f'{column}_median_14'] = median_arr
      mean_arr = []
      median_arr = []
  new_train_x = pd.concat([new_train_x, target], axis=0)
train_x_moving_14 = new_train_x.drop(raw_cols_tr, axis=1)
new_test_x = pd.DataFrame()
case_list = copy_test_x['Case'].unique()
for i in case_list:
  target = copy_test_x[copy_test_x['Case'] == i]
  mean_arr = []
  median_arr = []
  pbar = (target.columns[:])
  for column in pbar:
      column_list = target[column].to_list()
      for i in range(14):
        if((i) == (len(column_list))):
            break
        mean_arr.append(column_list[i])
        median_arr.append(column_list[i])
      for i in range(14, len(column_list)):
          mean_arr.append(float(np.mean(column_list[i-14:i])))
          median_arr.append(float(np.median(column_list[i-14:i])))
      target[f'{column}_mean_14'] = mean_arr
      target[f'{column}_median_14'] = median_arr
      mean_arr = []
      median_arr = []
  new_test_x = pd.concat([new_test_x, target], axis=0)
test_x_moving_14 = new_test_x.drop(raw_cols_te, axis=1)
train_x = pd.concat([train_x, train_x_moving_14], axis=1)
test_x = pd.concat([test_x, test_x_moving_14], axis=1)
train_x_fil = train_x.iloc[:,1:34]
test_x_fil = test_x.iloc[:,1:34]
def LPF(series, low, order=1):
    b, a = butter(
                  N = order,
                  Wn = low,
                  btype = 'low',
                  )
    lpf_series = lfilter(b, a, series)
    return lpf_series
new_train_x = pd.DataFrame()
case_list = train_x_fil['Case'].unique()
for i in case_list:
  target = train_x_fil[train_x_fil['Case'] == i]
  train_merge = LPF(target,0.1,1)
  train_merge = pd.DataFrame(train_merge)
  new_train_x = pd.concat([new_train_x, train_merge], axis=0)
new_train_x = new_train_x.add_suffix('_LPF')
new_train_x = new_train_x.reset_index(drop = True)
train_x = pd.concat([train_x,new_train_x], axis = 1)
new_test_x = pd.DataFrame()
case_list = test_x_fil['Case'].unique()
for i in case_list:
  target = test_x_fil[test_x_fil['Case'] == i]
  test_merge = LPF(target,0.1,1)
  test_merge = pd.DataFrame(test_merge)
  new_test_x = pd.concat([new_test_x, test_merge], axis=0)
new_test_x = new_test_x.add_suffix('_LPF')
new_test_x = new_test_x.reset_index(drop = True)
test_x = pd.concat([test_x,new_test_x], axis = 1)


train_x.drop(['0_LPF'],axis = 1, inplace=True)
test_x.drop(['0_LPF'],axis = 1, inplace=True)

train_x.drop(train_x.filter(regex = 'Case'),axis=1, inplace=True)
test_x.drop(test_x.filter(regex = 'Case'),axis=1, inplace=True)

# Modeling

In [ ]:
train = pd.concat([train_x,train_y], axis = 1)
train_data = TabularDataset(train)
test_data = TabularDataset(test_x)

predictor = TabularPredictor(label='predicted_weight_g',  eval_metric='root_mean_squared_error',path='/content/drive/MyDrive/상추/models').fit(train_data, presets='high_quality',  ag_args_fit={'num_gpus': 0})
y_pred = predictor.predict(test_data)

y_pred = pd.DataFrame(y_pred, columns=['predicted_weight_g'])

test_target_path = '/content/drive/MyDrive/상추/test_target/*.csv'
all_target_list = sorted(glob.glob(test_target_path))
i=0
for test_path in all_target_list:
    submit_df = pd.read_csv(test_path)
    submit_df['predicted_weight_g'] = y_pred[i*28:i*28+28].values
    submit_df.to_csv(test_path, index=False)
    i+=1